
----

## FlappyBird



输入为根据图像提取的关键点 (鸟,拐角) 的坐标, 采用经典的 DQN 算法, 经过 380 步就已经能够收敛到比较好的结果了. 下图示例中的就是 380 迭代后的结果 (只截取了前30秒的数据, 实际持续了 5 分钟).

![](FlappyBird-v0/dqn_agent.gif)


## CartPole-v0


<video id="pelican-installation" class="video-js vjs-default-skin" controls
preload="auto" width="683" height="384" poster="FlappyBird-v0/training_rewards.png"
data-setup="{}">
<source src="CartPole-v1/carpole_training.mp4" type='video/mp4'>
</video>

<script src="http://vjs.zencdn.net/4.0/video.js"></script>


<video id="video" controls="" preload="none" poster="FlappyBird-v0/training_rewards.png">
      <source id="mp4" src="CartPole-v1/carpole_training.mp4" type="video/mp4">
</video>